In [1]:
# Proces szkolenia modeli

# Zastrzeżenie
# Nie zalecam przeprowadzać pełnego procesu szkolenia na własnym sprzęcie, trening modelu ma bardzo duże wymagania sprzętowe
# Możliwe jest jednak przeprowadzenie poglądowego procesu szkolenia na słabszym sprzęcie, ale dopiero po zmianie zmiennej
#   w klasie Config z 'self.set_size = [90, 15, 20]' na 'self.set_size = [2, 2, 2]'
# Biblioteka TensorFlow wykorzystuje cały dostepny RAM, nawet jeżeli uczy się na bardzo małym zbiorze
# Dla 2 skanów jedna epoka uczenia trwa ok. 3-4 minuty

In [2]:
import os
import gc
import io
import math
import gzip
from datetime import datetime, timedelta
import numpy as np
import nibabel as nib
from tensorflow import data, cast, squeeze, math, reduce_mean
import tensorflow.keras.backend as K
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv3D, Conv3DTranspose, MaxPooling3D, concatenate, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, LearningRateScheduler, Callback
from tensorflow.keras.losses import BinaryCrossentropy

np.set_printoptions(suppress=True, precision=8)
# Ustaw poziom logowania na ERROR
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = INFO, 1 = WARNING, 2 = ERROR, 3 = FATAL
os.environ['XLA_FLAGS'] = '--xla_cpu_fast_math_honor_infs=false --xla_cpu_fast_math_honor_nans=false --xla_hlo_profile=false'

In [3]:
class Config():
    """
    Zbiór zmiennych globalnych i podlegająych zmianom w trakie procesu szkolenia.
    """

    def __init__(self, ax=0, env='kaggle'):
        # Oś podziału danych - 0, 1, 2
        self.ax = ax
        # Długość treningu
        self.train_epochs = 1
        # Jeżeli doszkalam model to muszę wpsiać na ilu epok model jest już przeszkolony
        self.previous_epochs = 0 
        # Liczba części na którą dzieli skan
        self.number_of_subset = [5, 5, 6][self.ax]
        # Liczba danych biorących udział w procesie szkolenia
        # self.set_size = [90, 15, 20] # train, val, test - max 90, 15, 20
        # Poglądowe szkolenie
        self.set_size = [2, 2, 2]
        # Liczba filtrów w pierwszej warstwie 
        self.filter_size_start = 32
        # Nazwa modelu
        self.name_output = f"K_{self.ax}_{str(self.filter_size_start)}_NFBS-20_{str(int([48, 48, 32][self.ax]))}_{str('UO-')}{self.previous_epochs + self.train_epochs}.weights.h5"
        
            # Nie podlega zmianom
        # Wybór środowiska
        self.env = env
        self.lr = 1e-5
        self.seed = 123
        self.batch_size = 1
        # Rozmiar jednej części
        self.img_size = [[48, 256, 192], [256, 48, 192], [256, 256, 32]][self.ax]
        # Liczba filtrów w sieci neuronowej, [32, 64, 128, 256, 512]
        self.filter_size = [self.filter_size_start * 2**x for x in range(5)]
        # Liczba iteracji w epoce
        self.subset_size = [self.number_of_subset * x for x in self.set_size]
        # Poziom informacji zwrotnych podczas treningu, kolejno dla Early, Reduce, Checkpoint, Fit
        # 0 - brak jakichkolwiek informacji w konsoli
        # 1 - w konsoli będzie wyświetlany pasek postępu dla każdej epoki
        # 2 - w konsoli będzie wyświetlany pasek postępu dla każdego batcha w epoce
        self.verbose = [1, 1, 1, 1]
        # Określa typ wczytywanych danych, standardowo float64 
        self.dtype = np.float32

        if self.env == 'kaggle':
            self.dataset_path = '/kaggle/input/nfbs-dataset-20/NFBS_Dataset_20'
            self.path_end = '.nii'
        elif self.env == 'laptop':
            self.dataset_path = 'NFBS_Dataset_20'
            self.path_end = '.nii.gz'



class DataNFBS():
    """
    Klasa wczytująca i przetwarzająca dane.
    Zwraca dane w formacie 'dataset' biblioteki TensorFlow, gotowe do uczenia.

    Metody
    ------
    subset_scope
        Zwraca zakresy podziału skanu na częsci
    normalization
        Wykonuje normalizację
    extract_gz
        Rozpakowanie pliku
    read_nii_file
        Przekształca dane z formatu .nii do listy
    read_dataset
        Zapisuje dane w formacie 'dataset'
    read_data
        Przekierowuje wczytanie wybranych podzbiorów
    """

    def __init__(self, config):
        self.config = config
        self.path = ['train', 'val', 'test']


    def subset_scope(self, ax):
        if ax == 2:
            ranges = [(8, 40), (37, 69), (66, 98), (95, 127), (124, 156), (153, 185)]
        elif ax == 1:
            ranges = [(0, 48), (45, 93), (90, 138), (135, 183), (180, 228)]
        elif ax == 0:
            ranges = [(28, 76), (73, 121), (118, 166), (163, 211), (208, 256)]

        return ranges


    def normalization(self, data):
        # Wszystko powyżej 600 na 600
        maxx = 600
        data = np.where(data > maxx, maxx, data)
        # Dzielimy przez 600, sprowadzamy do zakresu [0, 1]
        data = data / maxx

        return data
    

    def extract_gz(self, file_path):
        # Rozpakowanie pliku .gz
        with gzip.open(file_path, 'rb') as f_in:
            file_content = f_in.read()
        
        # Wczytanie danych z rozpakowanego strumienia
        file_stream = io.BytesIO(file_content)
        img = nib.Nifti1Image.from_bytes(file_stream.getvalue())   

        return img


    def read_nii_file(self, file_path, data):
        # jeśli wczytujemy plik na laptopie to musimy go najpierw rozpakować a dopiero potem wczytać
        if self.config.env == 'laptop':
            img = self.extract_gz(file_path)
        # jeśli wczytujemy plik z kaggle to jest on już rozpakowany, od razu możemy wczytać
        elif self.config.env == 'kaggle':
            img = nib.load(file_path)

        # img to wewnętrzny typ nibabel, wyciągamy dane i konwertujemy do określonego typu
        # scan to czyste dane [256, 256, 192]
        scan = img.dataobj[:, :, :].astype(self.config.dtype)

        # normalizacja, jeśli to nie jest maska
        if 'mask' not in file_path:
            scan = self.normalization(scan)

        # zapisujemy po kawałkach
        ranges = self.subset_scope(self.config.ax)
        for a, b in ranges:
            if self.config.ax == 2:
                data.append(scan[:, :, a:b])
            elif self.config.ax == 1:
                data.append(scan[:, a:b, :])
            elif self.config.ax == 0:
                data.append(scan[a:b, :, :])

        return data


    def read_dataset(self, pos):
        # position zawiera nazwę pozbioru
        position = self.path.index(pos)
        X_data = []
        y_data = [] # maska

        # pobieramy wszystkie nazwy plików z folderu podzbioru
        files = os.listdir(self.config.dataset_path + '/' + self.path[position])
        # sortujemy pliki po indeksie, który jest na pozycji 5:8
        files_sort = sorted(set([x[5:8] for x in files]))

        # wczytujemy X pierwszych plików
        for i in range(self.config.set_size[position]):
            X_data = self.read_nii_file(f"{self.config.dataset_path}/{self.path[position]}/NFBS_{str(files_sort[i])}_{self.path[position]}{self.config.path_end}", X_data)
            y_data = self.read_nii_file(f"{self.config.dataset_path}/{self.path[position]}/NFBS_{str(files_sort[i])}_mask_{self.path[position]}{self.config.path_end}", y_data)

        # zmieniam listę w której są 3 wymiarowe dane na 4 wymiarowe dane 
        X_data = np.stack(X_data, axis=0)
        y_data = np.stack(y_data, axis=0)
        print(X_data.shape)
        print(y_data.shape)

        # tworzę dataset
        # prefetch przygotowuje kolejne partie danych w tle, dzięki temu model nie musi czekać na załadowanie kolejnych danych w trakcie treningu
        # repeat powiela dane, umozliwiając trenowanie na więcej niż jednej eopoce
        dataset = data.Dataset.from_tensor_slices((X_data, y_data)).batch(self.config.batch_size).prefetch(data.AUTOTUNE).repeat()
        del X_data, y_data
        gc.collect()

        return dataset


    def read_data(self, status='learning'):
        # Wczytuje wszystkie podzbiory
        if status == 'all':
            train_dataset = self.read_dataset('train')
            val_dataset = self.read_dataset('val')
            test_dataset = self.read_dataset('test')
            return train_dataset, val_dataset, test_dataset
        # Wczytuje podzbiory potrzebne do treningu
        elif status == 'learning':
            train_dataset = self.read_dataset('train')
            val_dataset = self.read_dataset('val')
            return train_dataset, val_dataset, None
        # Wczytuje tylko podzbiór testowy, do oceny modelu
        elif status == 'test':
            test_dataset = self.read_dataset('test')
            return None, None, test_dataset
        else:
            print("Popraw.")



class UNetModel3D():
    """
    Architektura modelu
    """
    
    def __init__(self, config):
        self.config = config
        self.activation = 'relu'

    def conv_block(self, inp, filters):
        x = Conv3D(filters, (3, 3, 3), padding='same')(inp)
        x = BatchNormalization(axis=4)(x)
        x = Activation(self.activation)(x)
        x = Conv3D(filters, (3, 3, 3), padding='same')(x)
        x = BatchNormalization(axis=4)(x)
        x = Activation(self.activation)(x)
        return x

    def encoder_block(self, inp, filters):
        x = self.conv_block(inp, filters)
        p = MaxPooling3D(pool_size=(2, 2, 2))(x)
        # x - to ta długa strzałka w bok
        return x, p

    def decoder_block(self, inp, filters, concat):  # concat to od strzałki (x)
        x = Conv3DTranspose(filters, (3, 3, 3), strides=(2, 2, 2), padding='same')(inp)
        x = BatchNormalization(axis=4)(x)
        x = Activation(self.activation)(x)
        x = concatenate([x, concat])
        x = self.conv_block(x, filters)
        return x

    def create_model(self):
        inputs = Input((self.config.img_size[0], self.config.img_size[1], self.config.img_size[2], 1)) # 1 bo tylko odcień szarości

        x1, p1 = self.encoder_block(inputs, self.config.filter_size[0])
        x2, p2 = self.encoder_block(p1, self.config.filter_size[1])
        x3, p3 = self.encoder_block(p2, self.config.filter_size[2])
        x4, p4 = self.encoder_block(p3, self.config.filter_size[3])

        mid = self.conv_block(p4, self.config.filter_size[4])
        mid = Dropout(0.2)(mid)

        y1 = self.decoder_block(mid, self.config.filter_size[3], x4)
        y2 = self.decoder_block(y1, self.config.filter_size[2], x3)
        y3 = self.decoder_block(y2, self.config.filter_size[1], x2)
        y4 = self.decoder_block(y3, self.config.filter_size[0], x1)

        outputs = Conv3D(1, (1, 1, 1), activation='sigmoid')(y4)

        model = Model(inputs=inputs, outputs=outputs)

        print('Całkowita liczba parametrów: {:,}'.format(model.count_params()))

        return model
  


class ModelTrainer():
    """
    Klasa przygotowująca i przeprowadzająca proces szkolenia

    Metody
    ------
    linear_decay_lr
        Zmienia dynamicznie wartość stałej uczącej podczas treningu
    dice_coef
        Obliczanie metryki, która jest informacją zwrotną po każdej epoce
    LearningRateLogger
        Klasa - niestandardowy callback do zapisywania wartość stałej uczącej do logów
    train_model
        Przeprowadza proces szkolenia
    """

    def __init__(self, model, config):
        self.model = model
        self.config = config

        
    def linear_decay_lr(self, epoch, lr):
        # Jeżeli doszkalamy model to uwzględniamy na ilu epokach został juz przeszkolony
        epoch_new = self.config.previous_epochs + epoch

        if epoch_new < 2:
            return 1e-3
        elif epoch_new < 6:
            return 1e-4
        elif epoch_new < 10:
            return 8e-5
        elif epoch_new < 12:
            return 6e-5
        elif epoch_new < 16:
            return 4e-5
        elif epoch_new < 20:
            return 3e-5
        elif epoch_new < 50:
            initial_lr = 3e-5  
            final_lr = 1e-5    
            # Obliczenie nowej wartości learning rate na podstawie epoki
            new_lr = final_lr + (initial_lr - final_lr) * (1/30 * (50 - epoch_new))
            return new_lr
        elif epoch_new < 70:
            initial_lr = 1e-5
            final_lr = 4e-6
            new_lr = final_lr + (initial_lr - final_lr) * (1/20 * (70 - epoch_new))
            return new_lr
        else:
            return 4e-6

    
    def dice_coef(self, y_true, y_pred):
        # y_true - oryginalna maska, wartości 0 i 1
        # y_pred - maska z predykcji, wartości od 0 do 1

        # Dodajemy epsilon, żeby nie dzielić przez 0
        eps = K.epsilon()
        # greater sprawdza czy wartości są większe od 0.5, jeżeli tak to True, jeżeli nie to False
        # nastepnie wektor jest castowany, czyli wartości boolowskie są zmieniane na float32
        y_pred = cast(math.greater(y_pred, 0.5), 'float32')
        y_true = cast(y_true, 'float32')
        y_pred = squeeze(y_pred, axis=-1)
        # mnożenie żeby zostawić tylko te gdzie powielają się 1
        # axis=[1, 2, 3] oznacza po których osiach ma się sumować, 0 to wymiar przykładów uczących
        intersec = K.sum(y_true * y_pred, axis=[1, 2, 3])
        # liczenie współczynnika dice 
        dice = (2 * intersec + eps) / (K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3]) + eps)
        # dice to wektor ze współczynnikiem dla każdego przykładu uczącego

        # Zwraca średnią dla całego podzbioru
        return reduce_mean(dice)
    
    
    # Niestandardowy callback do logowania learning rate
    class LearningRateLogger(Callback):
        def on_epoch_end(self, epoch, logs=None):
            logs = logs or {}
            logs['learning_rate'] = K.get_value(self.model.optimizer.learning_rate.numpy())
    

    def train_model(self, train_dataset, val_dataset):
        # Wyświetl orientacyjny czas zakończenia szkolenia, jedna epoka trwa 504 sekundy na platformie Kaggle na kracie P100
        if self.config.env == 'kaggle':
            print(f'Start: {(datetime.now() + timedelta(hours=2)).strftime("%H:%M:%S")}')
            print(f'Koniec: {(datetime.now() + timedelta(hours=2) + timedelta(seconds=(504 * self.config.train_epochs))).strftime("%H:%M:%S")}')
        
        # optimizer do szybszego spadku gradientowego
        # funkcja straty jest używana do aktualizacji wag
        # metryka służy do monitorowania jakości modelu, informacyjnie dla nas
        self.model.compile(optimizer=Adam(learning_rate=self.config.lr), loss=BinaryCrossentropy(), metrics=[self.dice_coef])

        # callbacki są używane do wykonywania określonych działań na końcu epok, podczas trenowania modelu
        callbacks = [
                # Monitoruje wartość metryki (domyślnie val_loss) i przerywa trenowanie, jeśli ta metryka przestaje się poprawiać.
                # patience=10 - trenowanie zostanie przerwane, jeśli przez 10 kolejnych epok nie nastąpi poprawa wartości monitorowanej metryki
                # verbose=1 - zobaczysz w konsoli komunikaty, kiedy EarlyStopping zdecyduje się przerwać trenowanie.
            EarlyStopping(patience=10, verbose=self.config.verbose[0]),
                # Dynamicznie zmniejsza learning rate w przypadku, gdy metryka monitorowana (domyślnie na val_loss) przestaje się poprawiać.
                # factor=0.5 - Mnożnik, przez który zostanie pomnożona bieżąca szybkość uczenia. W tym przypadku learning rate zostanie zmniejszony do 10% swojej wartości.
                # patience=5 - Liczba epok bez poprawy, po których szybkość uczenia zostanie zmniejszona.
                # min_lr=0.00001 - Minimalna wartość learning rate, poniżej której nie zostanie obniżona.
            # ReduceLROnPlateau(factor=0.8, patience=2, min_lr=1e-06, verbose=self.config.verbose[1]),
                # Zapisuje wagi modelu na dysku podczas trenowania.
                # save_best_only=True - Zapisuje tylko te wagi modelu, które osiągnęły najlepszą wartość monitorowanej metryki (domyślnie na val_loss).
                # save_weights_only=True - Zapisuje tylko wagi modelu, a nie całą jego architekturę. Przydatne, gdy architektura modelu nie zmienia się podczas trenowania.
            ModelCheckpoint(self.config.name_output, verbose=self.config.verbose[2], save_best_only=True, save_weights_only=True),
                # Zapisuje logi trenowania do pliku CSV.
            CSVLogger('training_log.csv', append=True),
               # Dynamicznie zmienia learning rate w oparciu o zdefiniowaną funkcję.
            LearningRateScheduler(self.linear_decay_lr),
            self.LearningRateLogger()
        ]

        history = self.model.fit(
            train_dataset,
            # dane do walidacji, w specjalnym formacie
            validation_data = val_dataset,
            # liczba epok
            epochs=self.config.train_epochs,
            # Liczba określająca ile poditeracji na batchach ma się wykonać podczas epoki.
            # Ma sens w przypadku sekwencyjnym dostarczaniu danych (generator lub tf.data.Dataset) bo model nie wie ile dostanie danych i w którym momencie dostaje ponownie te same dane.
            # Metoda repeat na dataset w klasie DataNFBS będzie dostarczać dane w nieskończoność
            steps_per_epoch = int(self.config.subset_size[0] / self.config.batch_size),
            # To samo dla zbioru walidacyjnego.
            validation_steps = int(self.config.subset_size[1] / self.config.batch_size),
            # lista callbacków
            callbacks=callbacks,
            verbose=self.config.verbose[3]
        )

        return history

___

In [4]:
# ax - oś wzdłuż której dzielony jest skan, płaszczyzna czołowa --> 0 --> model czołowy
# env - środowisko w którym przeprowadzamy proces szkolenia
    # laptop - na własnym sprzęcie
    # kaggle - platforma Kaggle, należy odpowiednio wczytać pliki

config = Config(ax=0,
                env='laptop')

In [5]:
# Wybór danych które chcemy wczytać
# 'all' - wczytają się wszystkie dane
# 'learning' - wczytają się dane potrzebne do treningu
# 'test' - wczyta się tylko zbiór testowy, do ocena modelu, wykorzystywane w innym notebooku

train_dataset, val_dataset, test_dataset = DataNFBS(config).read_data('learning')
# learning wczytuje się 2-3 minuty na Kaggle

(10, 48, 256, 192)
(10, 48, 256, 192)
(10, 48, 256, 192)
(10, 48, 256, 192)


In [6]:
# Implementacja modelu, losowe wagi
model = UNetModel3D(config).create_model()
# Przygotowanie procesu szkolenia
model_trainer = ModelTrainer(model, config)

Całkowita liczba parametrów: 25,896,545


In [7]:
# Wczytanie wag jeżeli doszkalamy model
# model.load_weights('/kaggle/input/modele/K_0_32_NFBS-20_48_UO-24_O.weights.h5')

In [8]:
# Przeprowadzenie procesu szkolenia
history = model_trainer.train_model(train_dataset, val_dataset)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18s/step - dice_coef: 0.3435 - loss: 0.5519 
Epoch 1: val_loss improved from inf to 0.91469, saving model to K_0_32_NFBS-20_48_UO-1.weights.h5
10/10 ━━━━━━━━━━━━━━━━━━━━ 217s 21s/step - dice_coef: 0.3495 - loss: 0.5466 - val_dice_coef: 4.6025e-11 - val_loss: 0.9147 - learning_rate: 0.0010


In [9]:
# Zapisujemy ręcznie końcowy model, o ile to potrzebne
# model.save_weights('K_0_32_NFBS-20_48_UO-2444.weights.h5')